In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
%%time
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

CPU times: total: 0 ns
Wall time: 338 ms


200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [10]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)].reset_index(drop='index')
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
0,Зарубежные фильмы,09 Июн 23,Последняя сделка / The Last Deal (2023) WEB-DLRip от MegaPeer | Jaskier,1.46 GB,125,57
1,Зарубежные фильмы,09 Июн 23,"Детектив Найт: Мерзавец / Detective Knight: Rogue (2022) BDRip 1080p от селезень | D, P",16.51 GB,0,65
2,Зарубежные фильмы,09 Июн 23,"Детектив Найт: Искупление / Detective Knight: Redemption (2022) BDRip 1080p от селезень | D, P",10.18 GB,0,99
3,Зарубежные фильмы,09 Июн 23,"Детектив Найт: Независимость / Detective Knight: Independence (2023) BDRip 1080p от селезень | D, P",10.86 GB,0,96
4,Зарубежные фильмы,08 Июн 23,Последняя сделка / The Last Deal (2023) WEB-DL 1080p от New-Team | Jaskier,4.47 GB,187,53
5,Зарубежные сериалы,09 Июн 23,Укрытие / Бункер / Silo [01x01-07 из 10] (2023) WEB-DL 1080p | Невафильм,29.58 GB,83,938
6,Зарубежные сериалы,09 Июн 23,Переполненная комната / The Crowded Room [01x01-03 из 10] (2023) WEB-DL 1080p | Iyuno-SDI Group,11.36 GB,48,45
7,Зарубежные сериалы,08 Июн 23,Бойтесь ходячих мертвецов / Fear the Walking Dead [08x01-05 из 12] (2023) WEBRip 1080p от Kerob | L2,9.75 GB,7,44
8,Зарубежные сериалы,08 Июн 23,Перевал / Der Pass / Pagan Peak [S01-03] (2018-2023) WEB-DL 1080p | HDrezka Studio,52.84 GB,16,108
9,Зарубежные сериалы,08 Июн 23,Перевал / Der Pass / Pagan Peak [S01-03] (2018-2023) WEB-DLRip | HDrezka Studio,13.82 GB,34,89


In [22]:
id = [8,13]


my_df.loc[id, 'href']

8     http://rutor.info/torrent/858560/pereval_der-pass_pagan-peak-s01-03-2018-2023-web-dl-1080p-hdrezka-studio
13                                   http://rutor.info/torrent/928471/system-of-souls-2023-pc-repack-ot-fitgirl
Name: href, dtype: object